In [1]:
#imports 

%tensorflow_version 2.x
import tensorflow as tf 
from tensorflow import keras 

#Help-liberies 
import numpy as np
import matplotlib.pyplot as plt
import math as m
import random 



#we have 10 possible outcomes 
class_names = ['0','1','2','3','4','5','6','7','8','9']

#Nois on MNIST
For the experiment i want to put some noice over the Labels/
The idea is to misclassify some labels on purpose 

In [2]:
#The function takes the train labels and the amount of noise we want to apply 
#The return value is the train_labels with noise 
import random

def adding_noise(y_labels, percent):
  for y in y_labels:
    r = random.random()
    if r < percent:
      y = int(y + 1+ 9*random.random())%10
  return y_labels

#Creating a class Model with various attributes.

- the attribute layers describes how many hidden-layers the network whitch gets created has. \
- the attribute layersize describes the amount of neurons a hidden-layer has. \
- epochs describes how many epochs get fit into the model
- autoeochs is a boolean which, if it's true, regulates the epochs which are nececarry for the model to perfom well
- amount stands for the amount of itteration a model performs. So the average can be calculated 
- datasize is an integer which defines the size of the subset of the dataset   

In [3]:
[ ] 



class Model:
      model = keras.Sequential()

      def __init__(self, layers, layersize):

        self.layers = layers
        self.layersize = layersize
        #initializing the model
        self.model = keras.Sequential()
        self.model.add(keras.layers.Flatten(input_shape=(28,28)))#input layer
        for j in range(0, layers):
          self.model.add(keras.layers.Dense(layersize, activation="relu")) #hidden layers
        self.model.add(keras.layers.Dense(10,activation='softmax')) #output layer
        #compile model
        self.model.compile(optimizer= 'adam',loss= 'sparse_categorical_crossentropy',metrics=['accuracy'])  
        self.model.count_params()

      '''
      trains the model and returns a List of various data
      data[0] = the train loss after fitting the data 
      data[1] = the train accuracy after fitting the data  
      data[2] = the test loss after evaluating the model 
      data[3] = the test accuracy after evaluating the model  
      '''

      def train(self,epoch,datasize,train_images,train_labels,test_images, test_labels):
        data = np.zeros(4)
        print(datasize,epoch)
        history = self.model.fit(train_images[:datasize], train_labels[:datasize],epochs = epoch)
        train_loss, train_acc = history.history.values()
        data[0] = train_loss[epoch - 1]
        data[1] = train_acc[epoch - 1]
        test_loss, test_acc = self.model.evaluate(test_images, test_labels, verbose=1)
        data[2] = test_loss
        data[3] = test_acc
        print(self.model.summary())
        return data

      def getParameter(self):
        return self.model.count_params()




In [4]:
def trainModel(amount,min_layersize,max_layersize, layers, epochs, datasize, noise):
  a = max_layersize - min_layersize 
  data = np.zeros((5,a))

  #load dataset MNIST
  mnist = keras.datasets.mnist
  #spliting into testing and training
  (train_images, train_labels_without_noise), (test_images, test_labels) = mnist.load_data()
  #normalizing data
  train_images = train_images/255.0
  test_images = test_images/255.0
  #adding noise
  train_labels = adding_noise(train_labels_without_noise, noise)

  
  '''
  data[0][i] = the train loss after fitting the data in iterration i (with i layers) 
  data[1][i] = the train accuracy after fitting the data in iterration i (with i layers)  
  data[2][i] = the test loss after evaluating the model in iterration i (with i layers) 
  data[3][i] = the test accuracy after evaluating the model in iterration i (with i layers) 
  data[4][i] = the parameters of the model in iterration i (with i layers)
  '''

  for i in range(0,max_layersize - min_layersize):
    data_help = [0,0,0,0,0]
    for j in range(0,amount):
      print("I an currently in run ",i, " ===> The model has ", i+ min_layersize ,"neurons. ")
      print("amount =", amount) 
      myModel = Model(layers,i + min_layersize)
      history = myModel.train(epochs, datasize,train_images,train_labels,test_images, test_labels)
      param   = myModel.getParameter()
      data_help[0] = data_help[0] + history[0] 
      data_help[1] = data_help[1] + history[1]
      data_help[2] = data_help[2] + history[2]
      data_help[3] = data_help[3] + history[3]
      data_help[4] = data_help[4] + param

    data[0][i] = data_help[0]/amount
    data[1][i] = data_help[1]/amount
    data[2][i] = data_help[2]/amount
    data[3][i] = data_help[3]/amount
    data[4][i] = data_help[4]
  return data     

def custom_loss_function(y_true, y_pred):
   sum = 0.
   for i in range(0,len(y_true)):
     tf.autograph.experimental.set_loop_options(
        shape_invariants=[(sum, tf.TensorShape(None))]
     )
     sum = (sum + (y_true[i] - y_pred[i])**2)
   return tf.reduce_mean(tf.cast(1/len(y_true),tf.float32)*sum,axis=-1)

#Experimens

3 factors semm to have a big impact on the u shaped risk curve \
- The amount of Data. With more training Data the peak of the U-shape-Kurve tend to be more down the x-aches (More neurons are neccecary to get the effekt)
- The amount of noice, which was added to the training label. The peak sems to be way higher with higher label noise. 
- The amount of epochs. All models in this experiment are heaviliy overfitted. However the shape of the U seems to be better visible, if the amount of epochs is very high (around 200)  

In [ ]:

#experiment on label noise 
data = []


#effect of 30% noise
data_20 = trainModel(10,1,100,1,200,4000, 0.3)
data.append(data_20)

#effect of 15% noise 
data_10 = trainModel(10,1,100,1,200,4000, 0.15)
data.append(data_10)

#effect of 0% noise 
data_0 = trainModel(10,1,100,1,200,4000,0)
data.append(data_0)

In [ ]:
#plotting the experiment aboce

fig3, ax1 = plt.subplots(1, sharex = True)
fig3.set_size_inches(20,15)

ax1.set_title('kurves with different label noise') 
ax1.plot(data[0][2],  label= "30% noise", color = 'orange')
ax1.plot(data[1][2],  label= "15% noise",color ='blue')
ax1.plot(data[2][2],  label= "0% noise", color = 'green')

ax1.plot(data[0][0],color = 'orange')
ax1.plot(data[1][0],color = 'blue')
ax1.plot(data[2][0],color = 'green')


#legend
ax1.legend(loc='upper right')
plt.savefig('label_noise_low.png')

let's take a look at some higher noise level. 
We observed, that the Peak of the kurve is higher if the noise is higher. lets find out, what happens if we raise the noise level up to 50%

In [ ]:
#experiment on label noise 
data = []

#effect of 40% noise
data_30 = trainModel(10,1,100,1,200,4000,0.4)
data.append(data_30)

#effect of 60% noise 
data_50 = trainModel(10,1,100,1,200,4000,0.6)
data.append(data_50)

#effect of 80% noise 
data_70 = trainModel(10,1,100,1,200,4000,0.8)
data.append(data_70)

fig3, ax1 = plt.subplots(1, sharex = True)
fig3.set_size_inches(20,15)

ax1.set_title('kurves with different label noise') 
ax1.plot(data[0][2],  label= "40% noise", color = 'green')
ax1.plot(data[1][2],  label= "60% noise",color ='blue')
ax1.plot(data[2][2],  label= "80% noise", color = 'orange')

ax1.plot(data[0][0],color = 'green')
ax1.plot(data[1][0],color = 'blue')
ax1.plot(data[2][0],color = 'orange')


#legend
ax1.legend(loc='upper right')
plt.savefig('label_noise-high.png')

In [ ]:
#experiment on the effect of epochs

data_epochs = []
min_epochs = 20
max_epochs = 200
steps = 40


for i in range(min_epochs,max_epochs,steps):
  data_epochs.append(trainModel(10,1,100,1,i,4000,0.2))


In [ ]:

from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
viridis = cm.get_cmap('viridis', 12)
colors = viridis(np.linspace(0, 1, int(((max_epochs - min_epochs)/steps)+1)))

#plotting the experiment above

fig4, ax1 = plt.subplots(1, sharex = True)
fig4.set_size_inches(20,15)

ax1.set_title('kurves with different epochs') 
for i in range(0,len(data_epochs)):
  ax1.plot(data_epochs[i][0], color = colors[i], label = str(20 + i*40) + "epochs")
  ax1.plot(data_epochs[i][2], color = colors[i] )

#legend
ax1.legend(loc='upper right')
plt.savefig('epochs.png')



In [ ]:
#experiment on the influence of the amount of data
data_datasize = []
min_datasize = 1000
max_datasize = 11000
steps = 2000

#load dataset MNIST
mnist = keras.datasets.mnist
#spliting into testing and training
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
#adding noise to the traininglabels, so the effekt is more visible
train_labels = adding_noise(train_labels,0.2)

for i in range(min_datasize,max_datasize,steps):
  data_datasize.append(trainModel(4,1,30,1,100,i))

In [ ]:

from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
viridis = cm.get_cmap('viridis', 12)
colors = viridis(np.linspace(0, 1, int(((max_epochs - min_epochs)/steps)+1)))

#plotting the experiment above

fig4, ax1 = plt.subplots(1, sharex = True)
fig4.set_size_inches(20,15)

ax1.set_title('kurves with different datasize') 
for i in range(0,len(data_datasize)):
  ax1.plot(data_datasize[i][1], color = colors[i] )



#legend
ax1.legend(loc='upper right')
plt.savefig('data_size.png')
